In [1]:
"""
Read VIIRS active fire data
"""

# Import packages
import os, time, glob
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import h5py
import pyproj
import xarray as xr
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from affine import Affine
from osgeo import gdal, gdal_array, gdalconst, osr

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/'

# File path information
print("Success")

Success


In [16]:
# Function to list files in a directory
def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)

# Testing for one fire
fire_id = 'FIRED_3518'

# Get a list of .nc files
nc_files = list_files(os.path.join(maindir, f'data/spatial/raw/VIIRS/{fire_id}'), "*.nc", recursive=True)

# Seperate into VNP14 and VJ114, VNP03/VJ103

vnp = [f for f in nc_files if 'VNP14' in os.path.basename(f)] # VNP14IMG (active fire)
vnp03 = [f for f in nc_files if 'VNP03' in os.path.basename(f)] # VNP03IMG (geolocation)

vji = [f for f in nc_files if 'VJ114' in os.path.basename(f)] # VJI14IMG (active fire)
vji03 = [f for f in nc_files if 'VJ103' in os.path.basename(f)] # VJI14IMG (geolocation)

print(f'VNP14IMG: {len(vnp)} granules')
print(f'VNP03IMG: {len(vnp03)} granules')

print(f'VJI14IMG: {len(vji)} granules')
print(f'VJI03IMG: {len(vji03)} granules')


VNP14IMG: 17 granules
VNP03IMG: 17 granules
VJI14IMG: 16 granules
VJI03IMG: 16 granules


In [18]:
vnp[0]
vnp03[0]

'/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/raw/VIIRS/FIRED_3518/VNP03IMG/VNP03IMG.A2021240.2130.002.2021264132733.nc'

In [20]:
# Library to work with netCDF files
from netCDF4 import Dataset

# Open a .nc file ("file_name")
ds = Dataset(vnp[0], 'r', format = 'NETCDF4')
ds03 = Dataset(vnp03[0], 'r', format = 'NETCDF4')

print(ds)
print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print(ds03)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    VNP02IMG: VNP02CCIMG.A2021242.2048.002.2022348190854.nc
    VNP03IMG: VNP03IMG.A2021242.2048.002.2021264144156.nc
    VNP02MOD: VNP02CCMOD.A2021242.2048.002.2022348190854.nc
    VNP02GDC: VNP02GDC.A2021242.2048.002.2022267115438.nc
    ProcessVersionNumber: 3.1.9
    ExecutableCreationDate: Oct 31 2023
    ExecutableCreationTime: 17:46:00
    SystemID: Linux minion20055 5.4.0-1093-fips #103-Ubuntu SMP Thu Feb 8 14:02:37 UTC 2024 x86_64
    Unagg_GRingLatitude: 38.395008,43.871014,23.046049,18.644436
    Unagg_GRingLongitude: -137.594040,-101.392303,-99.409630,-128.482117
    NorthBoundingCoordinate: 43.8761
    SouthBoundingCoordinate: 18.644436
    EastBoundingCoordinate: -99.40963
    WestBoundingCoordinate: -137.59404
    Satellite: NPP
    Day/Night/Both: Day
    FirePix: 1659
    DayPix: 41369600
    LandPix: 36033495
    NightPix: 0
    RangeBeginningDate: 2021-08-30
    WaterPix: 5336105
  

In [ ]:

def read_geolocation_data(geo_file_path):
    with Dataset(geo_file_path, 'r') as geo_nc:
        print(geo_nc.groups.keys())  # List groups to see where the data is
        geo_group = geo_nc.groups['geolocation_data']  # Access the geolocation data group
        latitude = geo_group.variables['latitude'][:]  # Read latitude data
        longitude = geo_group.variables['longitude'][:]  # Read longitude data
        print(f"Latitude shape: {latitude.shape}")
        print(f"Longitude shape: {longitude.shape}")
        return latitude, longitude

latitude, longitude = read_geolocation_data(ds03)

In [28]:
# Explore the geolocation data:
print(ds03.groups.keys())  # List groups to see where the data is

dict_keys(['scan_line_attributes', 'geolocation_data', 'navigation_data'])


In [30]:
geo_da = ds03.groups['geolocation_data']  # Access the geolocation data group
latitude = geo_da.variables['latitude'][:]  # Read latitude data
longitude = geo_da.variables['longitude'][:]  # Read longitude data
print(f"Latitude shape: {latitude.shape}")
print(f"Longitude shape: {longitude.shape}")

Latitude shape: (6464, 6400)
Longitude shape: (6464, 6400)


In [25]:
day_night_flag = ds.getncattr('DayNightFlag')
print(day_night_flag)

Day


In [23]:
# Print a list of variables in file
print(list(ds.variables))
print("\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
print(list(ds03.variables))

['FP_AdjCloud', 'FP_AdjWater', 'FP_MAD_DT', 'FP_MAD_T4', 'FP_MAD_T5', 'FP_MeanDT', 'FP_MeanRad13', 'FP_MeanT4', 'FP_MeanT5', 'FP_Rad13', 'FP_SolAzAng', 'FP_SolZenAng', 'FP_T4', 'FP_T5', 'FP_ViewAzAng', 'FP_ViewZenAng', 'FP_WinSize', 'FP_confidence', 'FP_day', 'FP_latitude', 'FP_line', 'FP_longitude', 'FP_power', 'FP_sample', 'algorithm QA', 'fire mask']

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

[]


In [13]:
ds.getncattr('PlatformShortName')

'SUOMI-NPP'

In [ ]:
print(ds.variables[''])

In [ ]:
print(ds.variables['FP_confidence'][:])

In [ ]:
# Extract the acquisition date and time from the NetCDF file
start_time_str = ds.getncattr('PGE_StartTime')
acq_datetime = datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S.%f')
# Calculate Julian Day
julian_day = acq_datetime.timetuple().tm_yday

print(f'Acquisition date/time: {acq_datetime}')
print(f'Acquisition Julian Day: {julian_day}')

In [ ]:
# Extract fire pixel information
latitudes = ds.variables['FP_latitude'][:]
longitudes = ds.variables['FP_longitude'][:]
fire_radiative_power = ds.variables['FP_power'][:]
fire_confidence = ds.variables['FP_confidence'][:]
day_night = ds.variables['FP_day'][:]

# Create a DataFrame with the fire pixel data
fire_data = pd.DataFrame({
    'latitude': latitudes,
    'longitude': longitudes,
    'fire_radiative_power': fire_radiative_power,
    'fire_confidence': fire_confidence,
    'day_night': day_night,
    'acq_datetime': acq_datetime,
    'acq_julian_day': julian_day
})

fire_data.head()

In [ ]:
print(len(fire_data))

In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a Folium map centered around the mean latitude and longitude
m = folium.Map(location=[fire_data['latitude'].mean(), fire_data['longitude'].mean()], zoom_start=6)

# Add a marker cluster to the map
marker_cluster = MarkerCluster().add_to(m)

# Add markers to the map
for idx, row in fire_data.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f'FRP: {row["fire_radiative_power"]}',
        icon=folium.Icon(color='red')
    ).add_to(marker_cluster)

# Display the map
m.save('fire_radiative_power_map.html')
m